uv 0.9.5

## Question 2
Use uv to install Scikit-Learn version 1.6.1

What's the first hash for Scikit-Learn you get in the lock file?

Include the entire string starting with sha256:, don't include quotes

sha256:3faa5c39054b2f03ca547da9b2f52fde67c06240c31853f306aea97f13647b55

## Models

In [ ]:
categorical = ['lead_source']
numeric = ['number_of_courses_viewed', 'annual_income']

df[categorical] = df[categorical].fillna('NA')
df[numeric] = df[numeric].fillna(0)

train_dict = df[categorical + numeric].to_dict(orient='records')

pipeline = make_pipeline(
    DictVectorizer(),
    LogisticRegression(solver='liblinear')
)

pipeline.fit(train_dict, y_train)

In [3]:
!wget "https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin"

--2025-10-28 01:17:52--  https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin [following]
--2025-10-28 01:17:53--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1,3K) [application/octet-stream]
Saving to: 'pipeline_v1.bin'

     0K .      

## Question 3

Write a script for loading the pipeline with pickle

In [7]:
import pickle
pipeline_file = 'pipeline_v1.bin'
with open(pipeline_file, 'rb') as f_in:
    dict_vectorizer, model = pickle.load(f_in)

C:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle est


Score this record:

In [8]:
customer = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

In [10]:
X = dict_vectorizer.transform([customer])
model.predict_proba(X)[0,1]

np.float64(0.5336072702798061)

## Question 4

Install FastAPI

Write FastAPI code for serving the model

In [65]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle

app = FastAPI()

class Client(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float

pipeline_file = 'pipeline_v1.bin'
with open(pipeline_file, 'rb') as f_in:
    dict_vectorizer, model = pickle.load(f_in)

@app.post("/predict")
async def root(client_data: Client):
    client_dict = client_data.dict()
    X = dict_vectorizer.transform([client_dict])
    result = model.predict_proba(X)[0, 1]
    return result

FileNotFoundError: [Errno 2] No such file or directory: 'pipeline_v1.bin'

In [46]:
!pip install requests


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
import requests
requests.post("http://127.0.0.1:9999/predict")


<Response [422]>


Now score this client using requests:

In [49]:
url = "http://127.0.0.1:9999/predict"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}
requests.post(url, json=client).json()

0.5340417283801275

## Docker

FROM python:3.13.5-slim-bookworm

WORKDIR /code

COPY pipeline_v2.bin .


* agrigorev/zoomcamp-model:2025

## Question 5

Download the base image agrigorev/zoomcamp-model:2025.

You can easily make it by using docker pull command.

So what's the size of this base image?

In [54]:
!pip install docker


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [57]:
!docker image pull agrigorev/zoomcamp-model:2025

2025: Pulling from agrigorev/zoomcamp-model
816e9c0578b5: Pulling fs layer
66cd357cf7a1: Pulling fs layer
19f81082f6f0: Pulling fs layer
760c2b9cc6ee: Pulling fs layer
799df5177ae2: Pulling fs layer
760c2b9cc6ee: Download complete
816e9c0578b5: Download complete
66cd357cf7a1: Download complete
799df5177ae2: Download complete
19f81082f6f0: Download complete
19f81082f6f0: Pull complete
760c2b9cc6ee: Pull complete
816e9c0578b5: Pull complete
66cd357cf7a1: Pull complete
799df5177ae2: Pull complete
Digest: sha256:14d79fde0bbf078eb18c99c2bd007205917b758ec11060b2994963a1e485c2ae
Status: Downloaded newer image for agrigorev/zoomcamp-model:2025
docker.io/agrigorev/zoomcamp-model:2025


In [61]:
!docker images


REPOSITORY                 TAG                       IMAGE ID       CREATED         SIZE
agrigorev/zoomcamp-model   2025                      14d79fde0bbf   6 days ago      181MB
hh_auto_apply-frontend     latest                    f3a439460df2   2 weeks ago     81.7MB
hh_auto_apply-worker       latest                    3257ed10e14c   2 weeks ago     2.17GB
hh_auto_apply-flower       latest                    d916ae3886b7   2 weeks ago     2.17GB
hh_auto_apply-beat         latest                    7286e2f0d3c1   2 weeks ago     2.17GB
hh_auto_apply-api          latest                    3c8335a384ce   2 weeks ago     2.17GB
mcp/desktop-commander      latest                    7308759d4d03   3 weeks ago     671MB
redis                      7-alpine                  00fae95347ea   3 weeks ago     60.7MB
postgres                   15-alpine                 6bd113a3de32   3 weeks ago     390MB
bybiloader-crypto-app      latest                    793d65779338   4 weeks ago     1.76GB
bybi

## Dockerfile

Now create your own Dockerfile based on the image we prepared.

It should start like that:

> FROM agrigorev/zoomcamp-model:2025
>
> #add your stuff here
Now complete it:

- Install all the dependencies from pyproject.toml
- Copy your FastAPI script
- Run it with uvicorn

FROM agrigorev/zoomcamp-model:2025

WORKDIR /code

COPY pyproject.toml .

COPY server2.py .

RUN pip install --no-cache-dir .

EXPOSE 9999

CMD ["uvicorn", "server2:app", "--host", "0.0.0.0", "--port", "9999"]

## Question 6

Let's run your docker container!

After running it, score this client once again:


> url = "YOUR_URL"
>
> client = {
>
>    "lead_source": "organic_search",
>
>    "number_of_courses_viewed": 4,
>
>    "annual_income": 80304.0
>
> }
>
> requests.post(url, json=client).json()

What's the probability that this lead will convert?

In [66]:
url = "http://127.0.0.1:9999/predict"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}
requests.post(url, json=client).json()

0.9933071490756734

 # Publishing to Docker hub

In [ ]:
docker build -t homework-05 .
docker tag homework-05 ramires666/homework-05:1.1
docker push ramires666/homework-05:1.1